In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [39]:
source_path = 'D:\Python_Solution\Weather_project\Weather_Model_website\Data\Data_For_model\dataset_for_RNN.csv'

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Dell\AppData\Local\Temp\ipykernel_11568\2907910079.py:1: SyntaxWarning: invalid escape sequence '\P'
  source_path = 'D:\Python_Solution\Weather_project\Weather_Model_website\Data\Data_For_model\dataset_for_RNN.csv'


In [40]:
df = pd.read_csv(source_path)
df

,province,max,min,wind,rain,humidi,cloud,pressure,rain_norm
0,Bac Lieu,0.547619,0.666667,0.326531,6.9,0.870130,0.71,0.404255,0.069277
1,Bac Lieu,0.571429,0.666667,0.285714,0.5,0.805195,0.61,0.404255,0.005020
2,Bac Lieu,0.452381,0.633333,0.163265,16.7,0.883117,0.77,0.425532,0.167671
3,Bac Lieu,0.547619,0.633333,0.163265,2.2,0.818182,0.32,0.425532,0.022088
4,Bac Lieu,0.595238,0.666667,0.183673,0.0,0.753247,0.25,0.404255,0.000000
...,...,...,...,...,...,...,...,...,...
181472,Yen Bai,0.666667,0.700000,0.102041,7.3,0.727273,0.71,0.212766,0.073293
181473,Yen Bai,0.714286,0.766667,0.102041,0.3,0.623377,0.46,0.170213,0.003012
181474,Yen Bai,0.761905,0.833333,0.102041,0.0,0.532468,0.29,0.191489,0.000000
181475,Yen Bai,0.785714,0.866667,0.122449,0.0,0.558442,0.27,0.234043,0.000000


In [41]:
feats = ['max', 'min', 'wind','humidi','cloud','pressure']

In [42]:
scaler = MinMaxScaler()
scaler.fit(df[feats+['rain']])

MinMaxScaler()

In [43]:
df

,province,max,min,wind,rain,humidi,cloud,pressure,rain_norm
0,Bac Lieu,0.547619,0.666667,0.326531,6.9,0.870130,0.71,0.404255,0.069277
1,Bac Lieu,0.571429,0.666667,0.285714,0.5,0.805195,0.61,0.404255,0.005020
2,Bac Lieu,0.452381,0.633333,0.163265,16.7,0.883117,0.77,0.425532,0.167671
3,Bac Lieu,0.547619,0.633333,0.163265,2.2,0.818182,0.32,0.425532,0.022088
4,Bac Lieu,0.595238,0.666667,0.183673,0.0,0.753247,0.25,0.404255,0.000000
...,...,...,...,...,...,...,...,...,...
181472,Yen Bai,0.666667,0.700000,0.102041,7.3,0.727273,0.71,0.212766,0.073293
181473,Yen Bai,0.714286,0.766667,0.102041,0.3,0.623377,0.46,0.170213,0.003012
181474,Yen Bai,0.761905,0.833333,0.102041,0.0,0.532468,0.29,0.191489,0.000000
181475,Yen Bai,0.785714,0.866667,0.122449,0.0,0.558442,0.27,0.234043,0.000000


In [86]:
#df[feats+['rain_norm']] = scaler.transform(df[feats + ['rain']])

In [44]:
provinces = set(df.province)


In [45]:
def df_to_XY(df,window_size=10):
    X_train=[]
    y_train=[]
    X_test = []
    y_test = []
    X_val = []
    y_val = []
    fill = False
    for p in provinces:
        X = []
        y = []
        fts = df.loc[df['province'] == p][feats].values
        tg = df.loc[df['province'] == p]['rain'].values
        for i in range(window_size,len(tg)):
            features = fts[i-window_size:i-1].flatten() # np.concatenate([fts[i-window_size:i-1].flatten(), tg[i-window_size:i-1].flatten()])
            X.append(features)
            y.append(tg[i])
            
        X = np.array(X) 
        y = np.array(y)
        X_tr, X_t, y_tr, y_t = train_test_split(X, y, test_size=0.1)
        X_tr, X_v, y_tr, y_v = train_test_split(X_tr, y_tr, test_size=0.1)
        if not fill:
            X_train =  X_tr
            y_train =  y_tr
            X_test =  X_t
            y_test =  y_t
            X_val =  X_v
            y_val =  y_v
        else:            
            X_train =  np.append(X_train, X_tr,axis = 0)
            y_train =  np.append(y_train, y_tr,axis = 0)
            X_test =  np.append(X_test, X_t,axis = 0)
            y_test =  np.append(y_test, y_t,axis = 0)
            X_val =  np.append(X_val, X_v,axis = 0)
            y_val =  np.append(y_val, y_v,axis = 0)
        fill =True 
    return X_train, y_train, X_test, y_test, X_val, y_val

In [46]:
WINDOW = 15
X_train, y_train, X_test, y_test, X_val, y_val = df_to_XY(df,WINDOW)

In [11]:
X_train.shape

(146481, 84)

In [12]:
X_train

array([[0.69047619, 0.83333333, 0.10204082, ..., 0.84415584, 0.27      ,
        0.40425532],
       [0.76190476, 0.8       , 0.12244898, ..., 0.68831169, 0.3       ,
        0.36170213],
       [0.69047619, 0.8       , 0.12244898, ..., 0.80519481, 0.6       ,
        0.31914894],
       ...,
       [0.66666667, 0.8       , 0.30612245, ..., 0.77922078, 0.51      ,
        0.38297872],
       [0.64285714, 0.7       , 0.14285714, ..., 0.62337662, 0.23      ,
        0.42553191],
       [0.76190476, 0.86666667, 0.16326531, ..., 0.66233766, 0.39      ,
        0.38297872]])

In [47]:

#Reshaping(To add new dimensions)
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_val = np.reshape(X_val,(X_val.shape[0],X_val.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, ReLU, BatchNormalization

In [49]:
regressor = Sequential()

In [ ]:
#regressor = Sequential()
#Addinf the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units=50, return_sequences = True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
    
regressor.add(LSTM(units=50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
#output layer
regressor.add(Dense(units=1))

d:\Python_Solution\Weather_project\Weather_Model_website\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# regressor.add(LSTM(units=64, return_sequences = True, input_shape=(X_train.shape[1], 1)))
# regressor.add(Dropout(0.2))

# #regressor.add(Dense(units=64))
# regressor.add(ReLU())

# regressor.add(LSTM(units=32, return_sequences= True))
# #regressor.add(Dense(units=32))
# regressor.add(ReLU())

# regressor.add(LSTM(units=32, return_sequences= True))
# #regressor.add(Dense(units=32))
# regressor.add(ReLU())
# regressor.add(Dropout(0.2))
                      

# regressor.add(LSTM(units=32))
# #regressor.add(Dense(units=8))
# regressor.add(ReLU())

# #Output layer
# regressor.add(Dense(units=1, activation= 'linear'))





d:\Python_Solution\Weather_project\Weather_Model_website\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Input 0 of layer "lstm_4" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 1)

In [52]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError 
from tensorflow.keras.optimizers import Adam, schedules, SGD

In [53]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Tạo learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=100000,
    decay_rate=0.96
)

# Khởi tạo optimizer với lr_schedule
optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)

# Biên dịch mô hình với optimizer Adam và learning rate decay
regressor.compile(optimizer=optimizer, loss='mean_squared_error')

In [54]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = regressor.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=128, callbacks=[early_stopping])

Epoch 1/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 187s 158ms/step - loss: 119.2778 - val_loss: 113.6598
Epoch 2/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 205s 179ms/step - loss: 118.1663 - val_loss: 113.1961
Epoch 3/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 193s 168ms/step - loss: 120.9940 - val_loss: 113.2179
Epoch 4/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 255s 223ms/step - loss: 117.4629 - val_loss: 113.1944
Epoch 5/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 204s 178ms/step - loss: 115.9990 - val_loss: 112.2296
Epoch 6/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 207s 181ms/step - loss: 116.2426 - val_loss: 110.4127
Epoch 7/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 208s 182ms/step - loss: 109.3012 - val_loss: 99.0404
Epoch 8/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 200s 175ms/step - loss: 102.7516 - val_loss: 96.0188
Epoch 9/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 196s 171ms/step - loss: 101.1734 - val_loss: 95.8268
Epoch 10/100
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 225s 196ms/step - loss: 99.6750 - val_loss: 94.0885
Epoch 11/100
1145/1145 ━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
X_train.shape

NameError: name 'X_train' is not defined

In [ ]:
his = pd.DataFrame(history.history)

In [ ]:
his.head()

In [ ]:
import seaborn as sns
his.columns
history_loss = his[['loss', 'val_loss']]


fig,axes = plt.subplots(2,1,figsize=(14,8))
plt.subplot(2,1,1)
plt.title("Loss & Val Loss")
sns.lineplot(history_loss,palette="flare");

In [ ]:
train_pred = regressor.predict(X_train).flatten()
val_pred = regressor.predict(X_val).flatten()
test_pred = regressor.predict(X_test).flatten()

In [ ]:
pred = np.concatenate([train_pred,val_pred,test_pred])
df_pred = pd.DataFrame(np.concatenate([y_train,y_val,y_test]))
df_pred.columns=["actual"]
df_pred["predicted"] = pred

fig,axes = plt.subplots(2,1,figsize=(14,8),dpi=400)

plt.subplot(3,1,1)
plt.title("Train Results")
sns.lineplot(df_pred[:len(y_train)],alpha=0.8,palette="flare",linestyle=None);


plt.subplot(3,1,2)
plt.title("Validation Results")
sns.lineplot(df_pred[len(y_train):len(y_train)+len(y_val)],alpha=0.8,palette="flare",linestyle=None);

plt.subplot(3,1,3)
plt.title("Test Results")
sns.lineplot(df_pred[len(y_train)+len(y_val):],alpha=0.8,palette="flare",linestyle=None);

In [ ]:
import joblib

joblib.dump(le, "labelencoder.pkl") 
joblib.dump(scaler, "scaler.pkl") 
regressor.save('model.keras') 